In [5]:
# Disable warnings
import warnings
warnings.filterwarnings("ignore")

# Import pyplot from matplotlib
import matplotlib.pyplot as plt

# Import TensorFlow & Keras 
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image as image_preprocessing
import tensorflow.keras.backend as tf_backend 
tf_backend.set_image_data_format('channels_last')

# Import all other needed packages
import numpy as np
import csv
from PIL import Image

%matplotlib inline

In [7]:
# Reading data from the table

import csv

# Forming feature vectors dictionary
feature_vectors_dictionary = {}

with open('./output/keras_facenet_feature_vectors.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for i, row in enumerate(reader):
        if i == 0:
            continue
        
        person_id = int(float(row[0]))
        feature_vector = np.array(row[2:])
        feature_vector = feature_vector.astype(np.float)
        
        if person_id not in feature_vectors_dictionary:
            feature_vectors_dictionary[person_id] = [feature_vector]
            continue
            
        feature_vectors_dictionary[person_id].append(feature_vector)

dictionary_values = list(feature_vectors_dictionary.values()) 
        
# Finding minimal array size
min_array_length = int(1e10)
for value in dictionary_values:
    min_array_length = min(min_array_length, len(value))

# Forming tensorflow and numpy arrays
feature_vector_batches_np = []
feature_vector_batches_tf = []

for value in list(feature_vectors_dictionary.values()):
    value_np = np.array(value[:min_array_length])
    feature_vector_batches_np.append(value_np)
    feature_vector_batches_tf.append(tf.convert_to_tensor(value_np))
    
feature_vector_batches_tf = tf.stack(feature_vector_batches_tf)

In [8]:
# Model would consist of 3 dense layers of the same length as of feature vectors
def init_model():
    return keras.Sequential([
        layers.Dense(128, input_shape=(128,), activation="relu", name="dense_layer_1"),
        layers.Dense(128, activation="relu", name="dense_layer_2"),
        layers.Dense(128, name="dense_layer_3"),
    ])

# Initializing a model
model = init_model()

# Testing on a batch of inputs
test_output = model(feature_vector_batches_tf[0])
print('Test output shape:', test_output.shape)

# Providing a summary
model.summary()

Test output shape: (26, 128)
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_layer_1 (Dense)       (None, 128)               16512     
                                                                 
 dense_layer_2 (Dense)       (None, 128)               16512     
                                                                 
 dense_layer_3 (Dense)       (None, 128)               16512     
                                                                 
Total params: 49,536
Trainable params: 49,536
Non-trainable params: 0
_________________________________________________________________


In [ ]:
class Triplet_Loss(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()
    def call(self, y_true, y_pred):        
        log_y_pred = tf.math.log(y_pred)
        elements = -tf.math.multiply_no_nan(x=log_y_pred, y=y_true)
        return tf.reduce_mean(tf.reduce_sum(elements,axis=1))

In [10]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tfa.losses.TripletSemiHardLoss())

history = model.fit(
    feature_vector_batches_tf,
    epochs=5)

Epoch 1/5


ValueError: in user code:

    File "/home/dima/.conda/envs/newest/lib/python3.10/site-packages/keras/engine/training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "/home/dima/.conda/envs/newest/lib/python3.10/site-packages/keras/engine/training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/dima/.conda/envs/newest/lib/python3.10/site-packages/keras/engine/training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "/home/dima/.conda/envs/newest/lib/python3.10/site-packages/keras/engine/training.py", line 861, in train_step
        self._validate_target_and_loss(y, loss)
    File "/home/dima/.conda/envs/newest/lib/python3.10/site-packages/keras/engine/training.py", line 818, in _validate_target_and_loss
        raise ValueError(

    ValueError: Target data is missing. Your model was compiled with loss=<tensorflow_addons.losses.triple